In [31]:
import sys
import plotly.graph_objects as go
import pandas as pd
import sqlalchemy
from config import *
from urllib.parse import quote_plus
from ts_auth0 import TS_Auth
import requests
from datetime import datetime as dt
from datetime import timedelta

In [2]:
def getDf(ticker, start_date, end_date):
    # Create the database engine
    url = f'mysql://{DB_USER}:{quote_plus(DB_PASS)}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
    #print(url)
    engine = sqlalchemy.create_engine(url)

    query  = f"SELECT * FROM bars_1min WHERE ticker='{ticker}' AND "
    query += f"datetime >= '{start_date}' AND "
    query += f"datetime <= '{end_date}' "
    query += "ORDER BY datetime ASC"

    df = None
    with engine.connect() as conn:
        df = pd.read_sql(query, con = conn)

    #display(df)

    return df


,id,ticker,datetime,open,high,low,close,volume,open_interest
0,83787,RTYU23,2023-08-08 07:00:00,1956.9,1957.2,1956.7,1957.1,13,0
1,83788,RTYU23,2023-08-08 07:01:00,1957.2,1957.3,1956.5,1957.1,122,0
2,83789,RTYU23,2023-08-08 07:02:00,1957.0,1957.5,1956.6,1956.8,120,0
3,83790,RTYU23,2023-08-08 07:03:00,1956.6,1957.1,1955.8,1957.1,146,0
4,83791,RTYU23,2023-08-08 07:04:00,1957.0,1957.3,1956.7,1956.9,57,0


In [45]:

start_datetime = '2023-08-09T04:00:00Z'
end_datetime = '2023-08-09T15:00Z'

end_datetime = (dt.now() - timedelta(minutes=1)).strftime("%Y-%m-%dT%H:%M:00Z")
start_datetime = (dt.now() - timedelta(minutes=60)).strftime("%Y-%m-%dT%H:%M:00Z")
print(start_datetime, end_datetime)
df = getDf('SPY', start_datetime, end_datetime)

fig = go.Figure(data=[go.Candlestick(x=df['datetime'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])

fig.show()

2023-08-09T15:29:00Z 2023-08-09T16:28:00Z


In [29]:
start = "2023-06-24T00:00:00Z"
end = "2023-07-26T23:59:59Z"

#start = "2023-03-24T00:00:00Z"
#end = "2023-06-23T23:59:59Z"

ticker = "RTYU23"
ts = TS_Auth(API_KEY, API_SECRET_KEY)
ts.start_auth0()

access_token = ts.get_access_token()

#start_datetime, end_datetime = getDatetimeSpanForWeekNumber(2023, 28)
start_datetime = '2023-08-09T04:00:00Z'
end_datetime = '2023-08-09T04:20:00Z'

#start_datetime = '2023-08-09T14:20:00Z'
#end_datetime = '2023-08-09T14:30:00Z'

ticker = "RTYU23"
unit = "Minute"
url = f"https://sim-api.tradestation.com/v3/marketdata/barcharts/{ticker}?interval=1&unit={unit}&firstdate={start_datetime}&lastdate={end_datetime}"
#url = f"https://api.tradestation.com/v3/marketdata/barcharts/{ticker}?interval=1&unit={unit}&barsback=10"

#url = "https://sim-api.tradestation.com/v3/marketdata/barcharts/RTYU23?interval=1&unit=Minute&firstdate=2023-06-28T15:30:00Z&lastdate=2023-07-20T16:30:00Z"
print(url)  
headers = {'Authorization': f'Bearer {access_token}' }

response = requests.request("GET", url, headers=headers)
print(response.status_code)
json_data = response.json()
print(json_data)
for bar in json_data['Bars']:
    print(bar['TimeStamp'], bar['BarStatus'], bar['Open'], bar['High'], bar['Low'], bar['Close'], bar['TotalVolume'])

Attempting to get access token using previous refresh token pSJFzMb5j1rLboefn26X-18fMuHqyWUMkCrKS079x60Hf
Access token is expired, requesting a new one...
Got access token using previous refresh token
Access token already exists and is valid, skipping login
Access token is still valid, 1169.057609796524 seconds left until expiration
https://sim-api.tradestation.com/v3/marketdata/barcharts/RTYU23?interval=1&unit=Minute&firstdate=2023-08-09T04:00:00Z&lastdate=2023-08-09T04:20:00Z
200
{'Bars': [{'High': '1955.6', 'Low': '1955.4', 'Open': '1955.4', 'Close': '1955.6', 'TimeStamp': '2023-08-09T04:01:00Z', 'TotalVolume': '7', 'DownTicks': 1, 'DownVolume': 1, 'OpenInterest': '0', 'IsRealtime': False, 'IsEndOfHistory': False, 'TotalTicks': 7, 'UnchangedTicks': 0, 'UnchangedVolume': 0, 'UpTicks': 6, 'UpVolume': 6, 'Epoch': 1691553660000, 'BarStatus': 'Closed'}, {'High': '1955.6', 'Low': '1955.4', 'Open': '1955.6', 'Close': '1955.4', 'TimeStamp': '2023-08-09T04:02:00Z', 'TotalVolume': '4', 'DownT